## Feedback Loop Demo

This notebook demonstrates how feedback influences scoring weights.

In [1]:
from app.extensions.scoring_models import AdaptiveScoringProvider
from app.factories.logging_provider import CodeQualityLog, RecommendationLog
from app.utilities.feedback import FeedbackRepository
from app.utilities.metadata.logging.log_schemas import FeedbackLog

logs = {
    "code_quality": [
        CodeQualityLog(
            experiment_id="exp",
            round=0,
            symbol="m",
            lines_of_code=10,
            cyclomatic_complexity=5.0,
            maintainability_index=80.0,
            lint_errors=0,
        )
    ],
    "recommendation": [
        RecommendationLog(
            experiment_id="exp",
            round=0,
            symbol="m",
            file_path="m.py",
            line_start=0,
            line_end=0,
            recommendation="{}",
            context="ctx",
        )
    ],
}
provider = AdaptiveScoringProvider()
baseline = provider.score(logs)
baseline_weights = provider.weights.copy()
(baseline, baseline_weights)

({'maintainability_index': 80.0, 'cyclomatic_complexity': 5.0, 'recommendation_quality': 1.0}, {'maintainability_index': 1.0, 'cyclomatic_complexity': 1.0, 'recommendation_quality': 1.0})

Add feedback stressing maintainability.

In [2]:
FeedbackRepository.add_feedback(
    FeedbackLog("exp", 0, "user", "increase maintainability")
)
provider = AdaptiveScoringProvider()
updated = provider.score(logs)
updated_weights = provider.weights.copy()
(updated, updated_weights)

({'maintainability_index': 88.0, 'cyclomatic_complexity': 5.0, 'recommendation_quality': 1.0}, {'maintainability_index': 1.1, 'cyclomatic_complexity': 1.0, 'recommendation_quality': 1.0})

Notice how the maintainability weight increased after feedback.